# 멋진 챗봇 만들기

|평가문항|상세기준|
|---|---|
|1. 챗봇 훈련데이터 전처리 과정이 체계적으로 진행되었는가?|챗봇 훈련데이터를 위한 전처리와 augmentation이 적절히 수행되어 3만개 가량의 훈련데이터셋이 구축되었다.|
|2. transformer 모델을 활용한 챗봇 모델이 과적합을 피해 안정적으로 훈련되었는가?|과적합을 피할 수 있는 하이퍼파라미터 셋이 적절히 제시되었다.|
|3. 챗봇이 사용자의 질문에 그럴듯한 형태로 답하는 사례가 있는가?|주어진 예문을 포함하여 챗봇에 던진 질문에 적절히 답하는 사례가 제출되었다.|

# 패키지 및 라이브러리

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import re
import os
import io
import time
import random

import gensim

from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.bleu_score import sentence_bleu
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from collections import Counter
from konlpy.tag import Mecab

print(tf.__version__)

/opt/conda/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


2.4.1


# 데이터 다운로드

In [3]:
# data load
path_to_file = os.getenv('HOME')+'/aiffel/transformer_chatbot/ChatbotData.csv'
data = pd.read_csv(path_to_file)

In [4]:
data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [6]:
src = []
tgt = []
for s, t in zip(data['Q'], data['A']):
    src.append(str(s))
    tgt.append(str(t))

In [14]:
print('Q: ', src[0], '\n','Q 총 길이: ', len(src))
print('A: ', tgt[0], '\n','A 총 길이: ', len(tgt))

Q:  12시 땡! 
 Q 총 길이:  11823
A:  하루가 또 가네요. 
 A 총 길이:  11823


# 데이터 정제

In [15]:
mecab = Mecab()

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^0-9ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z?.!,]+", " ", sentence)

    sentence = sentence.strip()
    corpus = mecab.morphs(sentence)
    
    return corpus

Exception: Install MeCab in order to use it: http://konlpy.org/en/latest/install/